In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException    
import pandas as pd

In [2]:
driver = webdriver.Chrome(executable_path = './chromedriver.exe')
driver.maximize_window()

In [3]:
driver.get('https://www.iqair.com/us/world-air-quality')

**Hàm kiểm tra element có tồn tại không**

In [4]:
def check_element_exist(class_name):
    try:
        #aqi = driver.find_element_by_class_name('aqi-value__value')
        aqi = driver.find_element_by_class_name(class_name)
    except NoSuchElementException:
        return False
    return True

**Hàm kiểm tra text có phải là số không**

In [5]:
def is_number(text):
    for i in text:
        if i.isalpha():
            return False
    return True

# Lấy data mục "Air quality in the world"

In [6]:
driver.get('https://www.iqair.com/us/world-air-quality')
country_links = []
city_names = []
status_text = []
PM25 = []
PM10 = []
O3 = []
NO2 = []
SO2 = []
CO = []

board = driver.find_element_by_class_name('aqi-ranking')
countries = board.find_elements_by_tag_name('a')

#bo button SEE DETAIL o cuoi cung
countries = countries[:-1]
#countries = countries[:1]

for country in countries:
    #lay ra duong link dan den cac quoc gia trong top
    country_link = board.find_element_by_link_text(country.text).get_attribute('href')
    country_links.append(country_link)
    
for country_link in country_links:
    print('country: ', country_link)
    while True:        
        driver.get(country_link)
        #lay ra mot so thanh pho cua country
        if not check_element_exist('ranking__table'):
            break
            
        ranking_table = driver.find_element_by_class_name('ranking__table')
        cities = ranking_table.find_elements_by_tag_name('a')
        
        remain_city = False #kiem tra xem con thanh pho nao chua duoc duyet hay khong
        for city in cities:
            if city.text not in city_names:
                remain_city = True
                
                city_names.append(city.text)
                print('city name: ', city.text)
                city_link = city.get_attribute('href')
                driver.get(city_link)
        
                #lay ra muc do chat luong khong khi
                if not check_element_exist('aqi-status__text'):
                    status_text.append('NULL')
                    PM25.append('NULL')
                    PM10.append('NULL')
                    O3.append('NULL')
                    NO2.append('NULL')
                    SO2.append('NULL')
                    CO.append('NULL')
                else:
                    status = driver.find_element_by_class_name('aqi-status__text')
                    status_text.append(status.text)
                    print('Level: ', status.text)
                    #lay ra danh sach cac button chi so
                    button_board = driver.find_element_by_class_name('pollutant-history-type-buttons')
                    buttons = button_board.find_elements_by_tag_name('button')
                    #bo button AQI
                    buttons = buttons[1:]
                    for button in buttons:
                        print('button name: ', button.text)
                        #click vao tung button
                        button.click()
                        #lay chi so cuoi cung cua bieu do trong button
                        chart = driver.find_element_by_class_name('highcharts-series-group')
                        cols = chart.find_elements_by_tag_name('rect')
                        element_to_hover = cols[-1]
                        ActionChains(driver).move_to_element(element_to_hover).perform()
                        values = driver.find_elements_by_tag_name('tspan')
                        if len(values) == 1:
                            index_value = 0
                        else:
                            index_value = values[-2].text
                            if not is_number(index_value):
                                index_value = 0
                                
                        print(index_value)
                        
                        if button.text == 'PM2.5':
                            PM25.append(index_value)
                        elif button.text =='PM10':
                            PM10.append(index_value)
                        elif button.text == 'O3':
                            O3.append(index_value)
                            print('append')
                        elif button.text == 'NO2':
                            NO2.append(index_value)
                        elif button.text == 'SO2':
                            SO2.append(index_value)
                        elif button.text == 'CO':
                            CO.append(index_value)
                        
                    len_status_text = len(status_text)
                    if len(PM25) < len_status_text:
                        PM25.append('NULL')
                    if len(PM10) < len_status_text:
                        PM10.append('NULL')
                    if len(O3) < len_status_text:
                        O3.append('NULL')
                    if len(NO2) < len_status_text:
                        NO2.append('NULL')
                    if len(SO2) < len_status_text:
                        SO2.append('NULL')
                    if len(CO) < len_status_text:
                        CO.append('NULL')
                break
        #neu tat ca cac thanh pho da duoc duyet thi ket thuc
        if remain_city == False:
            break
print('finish')

country:  https://www.iqair.com/us/bangladesh
city name:  Dhaka, Dhaka
Level:  Very Unhealthy
button name:  PM2.5
182.1
city name:  Manikganj, Dhaka
Level:  Very Unhealthy
button name:  PM2.5
171
city name:  Savar, Dhaka
Level:  Unhealthy
button name:  PM2.5
119.1
city name:  Sreepur, Dhaka
Level:  Unhealthy
button name:  PM2.5
81.6
city name:  Narayanganj, Dhaka
Level:  Good
button name:  PM2.5
8
country:  https://www.iqair.com/us/pakistan
city name:  Rawalpindi, Punjab
Level:  Hazardous
button name:  PM2.5
276.3
city name:  Faisalabad, Punjab
Level:  Very Unhealthy
button name:  PM2.5
214.2
city name:  Islamabad, Islamabad
Level:  Unhealthy
button name:  PM2.5
132.8
city name:  Bahawalpur, Punjab
Level:  Very Unhealthy
button name:  PM2.5
154.6
city name:  Lahore, Punjab
Level:  Unhealthy
button name:  PM2.5
100.1
city name:  Sahiwal, Punjab
Level:  Unhealthy
button name:  PM2.5
126.5
city name:  Muridke, Punjab
Level:  Unhealthy
button name:  PM2.5
67.1
city name:  Abbottabad, Khybe

Level:  Good
button name:  PM2.5
9
button name:  PM10
12.3
button name:  O3
13.7
append
button name:  NO2
30.1
button name:  SO2
2.7
button name:  CO
340
city name:  Tartu, Tartumaa
Level:  Good
button name:  PM2.5
8.5
button name:  PM10
12.9
button name:  NO2
24.2
button name:  SO2
0
button name:  CO
350
city name:  Sinimaee, Ida-Virumaa
Level:  Good
button name:  PM2.5
4
button name:  PM10
6
button name:  SO2
0
city name:  Voore, Jogevamaa
Level:  Good
button name:  PM2.5
4
button name:  NO2
2.2
button name:  SO2
0.7
city name:  Sillamaee, Ida-Virumaa
Level:  Good
button name:  PM2.5
3.6
button name:  PM10
4
city name:  Kohtla-Jaerve, Ida-Virumaa
Level:  Good
button name:  PM2.5
3.5
button name:  PM10
4.3
button name:  NO2
5
button name:  SO2
0.8
city name:  Narva, Ida-Virumaa
Level:  Good
button name:  PM2.5
2.8
button name:  PM10
6.1
button name:  NO2
16
button name:  SO2
0.3
city name:  Parnu Jogi, Paernumaa
country:  https://www.iqair.com/us/sweden
city name:  Enkoeping, Uppsala 

In [7]:
data = {'City': city_names, 'Level': status_text, 'PM2.5': PM25, 'PM10': PM10, 'O3': O3, 'NO2': NO2, 'SO2': SO2, 'CO': CO}
df = pd.DataFrame(data)
df.to_csv("data.csv")

In [52]:
print(PM25)

['79.8', '68.3', '50.5', '44.6', '3', '105.9', '74.3', '72.1', '50.8', '90', '33', '31.9', '30.7', '30', '26.8', '36', '30.5', '18', '12', '12', '6.2', '5', '1', 'NULL', '10.2', '255', '200.5', '187.5', '172', '168', '162', '156.2', '153.6', '153.6', '152.9', '21.8', '21.2', '20.9', '18.6', '19', '18', '17', '16.2', '13.8', '10', '33', '63.2', '31.7', '31.3', '29.8', '27.3', '25.9', '25.1', '21.1', '14.7', '48', '63.5', '0.7', '0.8', '2.2', '1.4', '13', '8.6', '10.8', '8.3', '8', '8', '7.3', '7.4', '6.9', '6.4', '12', '11', '7.9', '7.1', '6.2', '6', '5.6', 'NULL', '8', '6.5', '5.7', '5', '4.7', '4.5', '4.2', 'NULL', '3.5', '3', '41.4', '32', '29.1', '26.4', '18.3', '18.5', '17.2', '14.6', '12.5', '11.4', '25.7', '11.4', '8.5', '7.6', '6.7', 'NULL', '5.5', 'NULL', 'NULL', '3.8', '36.7', '30.1', '26', '25', '25', '24', '23.8', '23.1', '21.5', '21', '11.6', '77.4', '51.2', '49.3', '48.4', '30.8', '13.8', '12.8', '14', '10.1']


In [53]:
print(PM10)

['NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '72', '52', '14', '16', 'NULL', '9', '19', 'NULL', 'NULL', 'NULL', '277', '225.9', 'NULL', '202', '248.7', '238', '224.4', 'NULL', '289', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '33', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '96.2', '139.4', '36.1', '31.8', 'NULL', '57', '69.4', 'NULL', '21.2', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '20.4', '56.3', '25.1', '10.5', '3.3', '16.7', '21.7', 'NULL', 'NULL', '9.1', '14.1', '13.3', '9.3', 'NULL', '6.4', '10.8', '6.4', 'NULL', '9.2', '8.6', 'NULL', '4.9', '8.1', 'NULL', 'NULL', 'NULL', '4', '3.5', '42.1', '39.7', '55.5', '35', '28.5', '13.8', 'NULL', '4.8', '14', '15.9', '3.7', '9.7', '13.5', '24.5', '24.3', '27.2', '3.1', '18.6', '17.4', '5.7', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '13', 'NULL', 'NULL', 'NULL', '379', '2', '30.6', '12.8', '82.9', '14', '18.6', '63.9', '55', 'NULL']


In [54]:
print(O3)

['NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '10.5', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '6', '11', '17', '58.8', '33.6', '21.6', '109.1', 'NULL', 'NULL', '45.9', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '56', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '4', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '18.1', 'NULL', '24.8', '16.4', 'NULL', 'NULL', 'NULL', '4.4', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '37.1', 'NULL', '32.4', 'NULL', 'NULL', '1.3', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '15', '12', '13', '14', 'NULL', 'NULL', '16', '40', 'NULL', 'NULL', 'NULL', 'NULL', '6', 'NULL', '4', '2', 'NULL', 'NULL']


In [55]:
print(NO2)

['NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '7', '7', '3', '6', 'NULL', '5', '2', 'NULL', 'NULL', '2.4', '58.8', '27.2', '108.4', '15.8', '40.4', '46.5', 'NULL', '8.8', '49.8', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '27.2', '6.6', '31.1', 'NULL', '1.9', '11.9', '16.1', 'NULL', 'NULL', '11.4', 'NULL', '37.7', '25.3', '3.3', 'NULL', '12.7', '4.9', '3.4', '9.9', '10.8', 'NULL', '4.9', '13.9', 'NULL', 'NULL', 'NULL', '18.9', '2.7', '40.3', '87.1', '89.2', '38.3', '40', '34.8', 'NULL', '20.9', '34', '40.5', '0.8', 'NULL', '7.3', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '2', '3', '3', '2', 'NULL', 'NULL', '25.1', '16', 'NULL', '3.8', 'NULL', '5.6', '3.8', '1.9', '3.8', '1.9', '5.6', 'NULL']


In [56]:
print(CO)

['NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '2', 'NULL', 'NULL', 'NULL', 'NULL', '2', 'NULL', 'NULL', 'NULL', 'NULL', '2660', '1410', '2590', '700', '960', '80', 'NULL', 'NULL', '1465', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '290', 'NULL', 'NULL', 'NULL', '280', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 0, 'NULL', 'NULL', 'NULL', 'NULL', '100', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '297.7', 'NULL', 'NULL', '4', 'NULL', 'NULL', 'NULL', '18.2', 'NULL', 'NULL', 'NU

# Lấy data một số quốc gia khác

In [8]:
country_links = []
city_names = []
status_text = []
PM25 = []
PM10 = []
O3 = []
NO2 = []
SO2 = []
CO = []

list_countries_str = 'japan china canada vietnam uk france germany thailand switzerland findland greece brazil russia philippines myanmar laos usa bulgaria malaysia peru serbia slovakia spain syria'
list_countries = list_countries_str.split()

for country in list_countries:
    #lay ra duong link dan den cac quoc gia trong top
    country_link = 'https://www.iqair.com/' + country
    country_links.append(country_link)
    
for country_link in country_links:
    print('country: ', country_link)
    while True:        
        driver.get(country_link)
        #lay ra mot so thanh pho cua country
        if not check_element_exist('ranking__table'):
            break
            
        ranking_table = driver.find_element_by_class_name('ranking__table')
        cities = ranking_table.find_elements_by_tag_name('a')
        
        remain_city = False #kiem tra xem con thanh pho nao chua duoc duyet hay khong
        for city in cities:
            if city.text not in city_names:
                remain_city = True
                
                city_names.append(city.text)
                print('city name: ', city.text)
                city_link = city.get_attribute('href')
                driver.get(city_link)
        
                #lay ra chi so AQI
                if not check_element_exist('aqi-status__text'):
                    status_text.append('NULL')
                    PM25.append('NULL')
                    PM10.append('NULL')
                    O3.append('NULL')
                    NO2.append('NULL')
                    SO2.append('NULL')
                    CO.append('NULL')
                else:
                    status = driver.find_element_by_class_name('aqi-status__text')
                    status_text.append(status.text)
                    #lay ra danh sach cac button chi so
                    button_board = driver.find_element_by_class_name('pollutant-history-type-buttons')
                    buttons = button_board.find_elements_by_tag_name('button')
                    #bo button AQI
                    buttons = buttons[1:]
                    for button in buttons:
                        print('button name: ', button.text)
                        #click vao tung button
                        button.click()
                        #lay chi so cuoi cung cua bieu do trong button
                        chart = driver.find_element_by_class_name('highcharts-series-group')
                        cols = chart.find_elements_by_tag_name('rect')
                        element_to_hover = cols[-1]
                        ActionChains(driver).move_to_element(element_to_hover).perform()
                        values = driver.find_elements_by_tag_name('tspan')
                        if len(values) == 1:
                            index_value = 0
                        else:
                            index_value = values[-2].text
                            if not is_number(index_value):
                                index_value = 0
                                
                        print(index_value)
                        
                        if button.text == 'PM2.5':
                            PM25.append(index_value)
                        elif button.text =='PM10':
                            PM10.append(index_value)
                        elif button.text == 'O3':
                            O3.append(index_value)
                        elif button.text == 'NO2':
                            NO2.append(index_value)
                        elif button.text == 'SO2':
                            SO2.append(index_value)
                        elif button.text == 'CO':
                            CO.append(index_value)
                        
                    len_status_text = len(status_text)
                    if len(PM25) < len_status_text:
                        PM25.append('NULL')
                    if len(PM10) < len_status_text:
                        PM10.append('NULL')
                    if len(O3) < len_status_text:
                        O3.append('NULL')
                    if len(NO2) < len_status_text:
                        NO2.append('NULL')
                    if len(SO2) < len_status_text:
                        SO2.append('NULL')
                    if len(CO) < len_status_text:
                        CO.append('NULL')
                    
                break
        #neu tat ca cac thanh pho da duoc duyet thi ket thuc
        if remain_city == False:
            break
print('finish')

country:  https://www.iqair.com/japan
city name:  Ube, Yamaguchi
button name:  PM2.5
30
button name:  PM10
8
button name:  NO2
9.4
button name:  SO2
7.9
city name:  Hiraidemachi, Tochigi
button name:  PM2.5
23
button name:  PM10
8
button name:  NO2
39.5
city name:  Minato, Wakayama
button name:  PM2.5
20
button name:  PM10
24
button name:  SO2
23.6
city name:  Mizusawaku Higashiodori, Iwate
button name:  PM2.5
19
button name:  PM10
4
button name:  NO2
26.3
button name:  SO2
2.6
city name:  Futago, Kanagawa
button name:  PM2.5
15
button name:  PM10
11
button name:  NO2
73.3
city name:  Tamana, Kumamoto
button name:  PM2.5
18
button name:  PM10
8
button name:  NO2
7.5
city name:  Hanazakicho, Chiba
button name:  PM2.5
17
city name:  Yumeimachi, Fukuoka
button name:  PM2.5
16
button name:  PM10
7
button name:  NO2
9.4
button name:  SO2
2.6
button name:  CO
229
city name:  Higashikoshigaya, Saitama
button name:  PM2.5
15
button name:  PM10
15
button name:  NO2
45.1
button name:  SO2
2.6
ci

23.7
city name:  Lampang, Lampang
button name:  PM2.5
44.5
button name:  PM10
19
button name:  O3
84
button name:  SO2
2.6
city name:  San Sai, Chiang Mai
button name:  PM2.5
39.9
city name:  Hua Hin, Prachuap Khiri Khan
button name:  PM2.5
39.7
city name:  Pai, Mae Hong Son
button name:  PM2.5
39.6
city name:  Ban Sang, Prachin Buri
button name:  PM2.5
38.6
country:  https://www.iqair.com/switzerland
city name:  Kloten / Balsberg, Zurich
button name:  PM2.5
41
button name:  PM10
14.9
button name:  O3
19.6
button name:  NO2
32.1
city name:  Bodio, Ticino
button name:  PM2.5
37
button name:  PM10
73
button name:  O3
3
button name:  NO2
63
button name:  SO2
16
city name:  Lumino, Ticino
button name:  PM2.5
31
button name:  PM10
39.5
button name:  NO2
34.1
city name:  Ecublens, Vaud
button name:  PM2.5
30.9
city name:  Visp, Valais
button name:  PM2.5
25
button name:  PM10
31
button name:  O3
24
button name:  NO2
39
city name:  Dubendorf, Zurich
button name:  PM2.5
22.6
button name:  PM10

4
city name:  Puerto Maldonado, Madre de Dios
button name:  PM2.5
2.3
city name:  Santa Anita, Lima
button name:  NO2
50.4
button name:  SO2
15.4
city name:  Villa Maria del Triunfo, Lima
button name:  SO2
12.9
button name:  CO
1
country:  https://www.iqair.com/serbia
city name:  Valjevo, Central Serbia
button name:  PM2.5
91.9
button name:  PM10
92.6
button name:  NO2
59.8
button name:  SO2
7.7
city name:  Cacak, Central Serbia
button name:  PM2.5
74.7
button name:  NO2
14.3
city name:  Uzice, Central Serbia
button name:  PM2.5
63.1
button name:  PM10
63.2
button name:  NO2
2.7
button name:  SO2
10
city name:  Novi Pazar, Central serb
button name:  PM2.5
55.7
button name:  PM10
56.6
button name:  O3
27.5
button name:  NO2
46.8
button name:  SO2
23.2
city name:  Nis, Central Serbia
button name:  PM2.5
48.1
button name:  PM10
16.3
button name:  O3
37.9
button name:  NO2
34.8
button name:  SO2
9.7
city name:  Pancevo, Central Serbia
button name:  PM2.5
30
button name:  PM10
35.2
button n

In [9]:
data = {'City': city_names, 'Level': status_text, 'PM2.5': PM25, 'PM10': PM10, 'O3': O3, 'NO2': NO2, 'SO2': SO2, 'CO': CO}
df = pd.DataFrame(data)
df.to_csv("data_1.csv")